In [ ]:
import torch
import json
import logging
import numpy as np
import itertools
import operator
from lstm_crf.dataset import Dataset
from lstm_crf.args import Config
from transformers import AutoTokenizer, AutoModel
from nltk.tokenize import word_tokenize, sent_tokenize

from seqlbtoolkit.text import split_overlength_bert_input_sequence
from seqlbtoolkit.data import merge_list_of_lists
from seqlbtoolkit.io import save_json

logger = logging.getLogger(__name__)

In [ ]:
with open('./data/MSIE/train.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [12]:
max_seq_length = 512

for data_id in range(len(data)):
    print(data_id)
    tks = data[f'{data_id}']['data']['text']
    tks_seq_list = [word_tokenize(sent) for sent in sent_tokenize(' '.join(tks))]
    seq_bert_len_list = [len(tokenizer.tokenize(' '.join(tks_seq), add_special_tokens=True)) for tks_seq in tks_seq_list]
    split_points = [0, len(tks_seq_list)]
    split_bert_lens = [sum(seq_bert_len_list[split_points[i]:split_points[i+1]]) for i in range(len(split_points)-1)]

    while (np.asarray(split_bert_lens) > max_seq_length).any():

        new_split_points = list()
        for idx, bert_len in enumerate(split_bert_lens):
            # print(idx, bert_len)
            if bert_len > max_seq_length:
                seq_bert_len_sub_list = seq_bert_len_list[split_points[idx]:split_points[idx+1]]
                seq_bert_len_sub_accu_list = list(itertools.accumulate(seq_bert_len_sub_list, operator.add))
                # print(f"bert len: {bert_len}, accu list: {seq_bert_len_sub_accu_list}")
                # try to separate sentences as evenly as possible
                split_offset = np.argmin((np.array(seq_bert_len_sub_accu_list) - bert_len / 2) ** 2)
                new_split_points.append(split_offset + split_points[idx] + 1)
                # print(f"split offset: {split_offset}")
        # print(f"new split points: {new_split_points}")

        split_points += new_split_points
        split_points.sort()

        split_bert_lens = [sum(seq_bert_len_list[split_points[i]:split_points[i+1]]) for i in range(len(split_points)-1)]
        # print(f"split bert lengths: {split_bert_lens}")

    split_tks_seq_list = [tks_seq_list[split_points[i]:split_points[i+1]] for i in range(len(split_points)-1)]



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62


In [11]:
tks = data['8']['data']['text']
tks_seq_list = [word_tokenize(sent) for sent in sent_tokenize(' '.join(tks))]
seq_bert_len_list = [len(tokenizer.tokenize(' '.join(tks_seq), add_special_tokens=True)) for tks_seq in tks_seq_list]
split_points = [0, len(tks_seq_list)]
split_bert_lens = [sum(seq_bert_len_list[split_points[i]:split_points[i+1]]) for i in range(len(split_points)-1)]

while (np.asarray(split_bert_lens) > max_seq_length).any():

    new_split_points = list()
    for idx, bert_len in enumerate(split_bert_lens):
        print(idx, bert_len)
        if bert_len > max_seq_length:
            seq_bert_len_sub_list = seq_bert_len_list[split_points[idx]:split_points[idx+1]]
            seq_bert_len_sub_accu_list = list(itertools.accumulate(seq_bert_len_sub_list, operator.add))
            print(f"bert len: {bert_len}, accu list: {seq_bert_len_sub_accu_list}")
            # try to separate sentences as evenly as possible
            split_offset = np.argmin((np.array(seq_bert_len_sub_accu_list) - bert_len / 2) ** 2)
            new_split_points.append(split_offset + split_points[idx] + 1)
            print(f"split offset: {split_offset}")
    print(f"new split points: {new_split_points}")

    split_points += new_split_points
    split_points.sort()

    split_bert_lens = [sum(seq_bert_len_list[split_points[i]:split_points[i+1]]) for i in range(len(split_points)-1)]
    print(f"split bert lengths: {split_bert_lens}")

split_tks_seq_list = [tks_seq_list[split_points[i]:split_points[i+1]] for i in range(len(split_points)-1)]

0 7887
bert len: 7887, accu list: [106, 186, 216, 267, 341, 415, 457, 496, 559, 597, 635, 663, 703, 722, 748, 786, 849, 891, 956, 1020, 1090, 1142, 1173, 1272, 1383, 1408, 1441, 1461, 1471, 1526, 1561, 1576, 1613, 1635, 1674, 1697, 1852, 1886, 1920, 1930, 1973, 2051, 2094, 2125, 2138, 2157, 2168, 2174, 2202, 2240, 2248, 2293, 2344, 2380, 2398, 2421, 2436, 2495, 2531, 2570, 2599, 2612, 2629, 2644, 2685, 2699, 2748, 2812, 2852, 2899, 2922, 2945, 2983, 2998, 3010, 3065, 3103, 3167, 3199, 3226, 3289, 3304, 3357, 3395, 3433, 3459, 3480, 3505, 3517, 3573, 3629, 3672, 3704, 3732, 3794, 3801, 3822, 3856, 3922, 4002, 4067, 4093, 4131, 4194, 4241, 4276, 4313, 4341, 4346, 4380, 4420, 4504, 4532, 4561, 4586, 4639, 4724, 4730, 4760, 4828, 4849, 4893, 4934, 4961, 5010, 5043, 5177, 5200, 5205, 5310, 5329, 5348, 5372, 5396, 5427, 5492, 5514, 5561, 5614, 5719, 5820, 5884, 5911, 5932, 5948, 5991, 6028, 6051, 6075, 6108, 6196, 6204, 6263, 6305, 6364, 6564, 6608, 6720, 6757, 6805, 6876, 6948, 6996, 7074, 

In [ ]:
from chemdataextractor.nlp.tokenize import ChemWordTokenizer

In [ ]:
cwt = ChemWordTokenizer()

In [ ]:
cwt.tokenize(' '.join(ori_tks))

In [ ]:
from chemdataextractor.doc import Paragraph

In [ ]:
para = Paragraph(' '.join(ori_tks))

In [ ]:
para.raw_sentences

In [ ]:
tks = para.tokens

In [ ]:
len(tks[0])

In [ ]:
len(merge_list_of_lists(tks))

In [ ]:
len(merge_list_of_lists([sent.tokens for sent in para.sentences]))

In [ ]:
para.sentences[0].end